In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [12]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using Polynomials
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using MLDataUtils
using MLDataPattern
using Test
using SparseArrays
using QRupdate
using AdaptiveTransportMap: vander, transform!, evaluate, ncoeff, optimize, negative_log_likelihood!, derivative
using Quadrature
using Cubature
using FiniteDiff
using Zygote

### 1D example

In [50]:
Nx = 1
Ne = 500
X = randn(Nx, Ne)
ens = EnsembleState(X)
m = 10
B = MultiBasis(CstProHermite(3), Nx)

idx = reshape([0; 1; 2; 3], (4,1))

coeff =  randn(size(idx,1))

C = MapComponent(m, Nx, idx, coeff)
R = C.I;

In [51]:
ForwardDiff.gradient(x->log_pdf(evaluate(C, x)), member())

LoadError: MethodError: no method matching member()
Closest candidates are:
  member(!Matched::EnsembleState{Nx,Ne}, !Matched::Int64) where {Nx, Ne} at /media/mat/HDD/TransportMap/src/DataAssimilation/ensemble.jl:81

In [52]:
function Gradient(x)
    return ForwardDiff.gradient(y->begin
                           y[end] = 0.0
                           return  R.f.f(y)
        end, x) + quadgk(t->ForwardDiff.gradient(
                  z->R.g(ForwardDiff.gradient(y->R.f.f(y), vcat(z[1:end-1],t))[end]),
            x), 0.0, x[end])[1] + vcat(zeros(size(x,1)-1), R.g(ForwardDiff.gradient(y->R.f.f(y), x)[end]))
end

Gradient (generic function with 1 method)

In [53]:
function Gradient_log_pdf(x)
    G = Gradient(x)
    Cx = evaluate(C, x)
end

Gradient_log_pdf (generic function with 1 method)

In [85]:
function evaluatef0(x)
    y = copy(x)
    y[end] = 0.0
    return C.I.f.f(y)
end

evaluatef0 (generic function with 1 method)

In [143]:
integrand(t,x) = C.I.g(ForwardDiff.gradient(y->C.I.f.f(y), vcat(x[1:end-1],t*x[end]))[end])

integrand (generic function with 1 method)

In [144]:
function Ct(x)
    lb = 0.0
    ub = 1.0
    prob = QuadratureProblem(integrand,lb,ub,x)
#     if size(x,1)>1
    out = evaluatef0(x) + x[end]*solve(prob,CubatureJLh(),reltol=1e-6,abstol=1e-6)[1]

    return out
end

Ct (generic function with 1 method)

In [145]:
x = member(ens,1)

1-element Array{Float64,1}:
 1.4888500163721123

In [146]:
evaluatef0(x)

0.4842460138490202

In [147]:
Ct(x)

1.8226408855720533

In [148]:
evaluate(C, X)

500-element Array{Float64,1}:
  1.822640885572053
 -0.1106099017787247
 -0.3334545288753537
  1.4342095500687038
 -0.8088972259660936
 -0.19418683976164047
 -0.5587887651493522
  1.2456300934917655
 -0.5813029086031765
  1.3079418618221277
  0.49177266568032685
  1.2622575502779165
  1.3082074026613686
  ⋮
  0.9623444305521173
  0.5591882644836959
 -0.22790998436271842
 -0.12132974211889375
  0.2791643259744656
 -2.004380882380656
 -0.5183170045751484
  0.6432072139247168
 -0.8327627994277031
  0.7180680827933583
 -1.622414056544571
 -0.9183342103507903

In [149]:
log_pdfCt(x) = log_pdf(Ct(x)) + log(C.I.g(ForwardDiff.gradient(z->C.I.f.f(z),x)[end]))

log_pdfCt (generic function with 1 method)

In [150]:
log_pdfCt(x)

-2.7326939160649277

In [158]:
grad_x_log_pdf(C, X)

500×1 Array{Float64,2}:
 -1.5601399881865656
 -0.020124448084395966
  0.24831494645778796
 -1.3019048606193167
  0.8615564950053465
  0.07854224436643495
  0.5342323103196984
 -1.1847991949449275
  0.5633785624503269
 -1.2234814418342665
 -0.6394025411116088
 -1.1951400049356424
 -1.2236459926252832
  ⋮
 -1.0022017448374185
 -0.6975635466597019
  0.11906919291278784
 -0.00761640842202263
 -0.4409414651209079
  2.3459968434679572
  0.482062094696478
 -0.7669259059429174
  0.8930532536654968
 -0.8259234781376762
  1.9069090512427576
  1.0060745313515818

In [157]:
ForwardDiff.gradient(log_pdfCt, x)

1-element Array{Float64,1}:
 -1.5601399881865656

In [153]:
grad_x_log_pdf(C, X)

500×1 Array{Float64,2}:
 -1.5601399881865656
 -0.020124448084395966
  0.24831494645778796
 -1.3019048606193167
  0.8615564950053465
  0.07854224436643495
  0.5342323103196984
 -1.1847991949449275
  0.5633785624503269
 -1.2234814418342665
 -0.6394025411116088
 -1.1951400049356424
 -1.2236459926252832
  ⋮
 -1.0022017448374185
 -0.6975635466597019
  0.11906919291278784
 -0.00761640842202263
 -0.4409414651209079
  2.3459968434679572
  0.482062094696478
 -0.7669259059429174
  0.8930532536654968
 -0.8259234781376762
  1.9069090512427576
  1.0060745313515818

In [128]:
evaluate(C, X)

500-element Array{Float64,1}:
  1.822640885572053
 -0.1106099017787247
 -0.3334545288753537
  1.4342095500687038
 -0.8088972259660936
 -0.19418683976164047
 -0.5587887651493522
  1.2456300934917655
 -0.5813029086031765
  1.3079418618221277
  0.49177266568032685
  1.2622575502779165
  1.3082074026613686
  ⋮
  0.9623444305521173
  0.5591882644836959
 -0.22790998436271842
 -0.12132974211889375
  0.2791643259744656
 -2.004380882380656
 -0.5183170045751484
  0.6432072139247168
 -0.8327627994277031
  0.7180680827933583
 -1.622414056544571
 -0.9183342103507903

In [163]:

  Nx = 2
  Ne = 100
  X = randn(Nx, Ne)
  ens = EnsembleState(X)
  m = 10
  B = MultiBasis(CstProHermite(3), Nx)

  idx = [0 0; 0 1; 1 0; 1 1; 1 2; 3 2]

  coeff =  randn(size(idx,1))

  C = MapComponent(m, Nx, idx, coeff)

  dxlogC = grad_x_log_pdf(C, X)

  function evaluatef0(x)
    y = copy(x)
    y[end] = 0.0
    return C.I.f.f(y)
  end

  integrand(t,x) = C.I.g(ForwardDiff.gradient(y->C.I.f.f(y), vcat(x[1:end-1],t*x[end]))[end])

  function Ct(x)
      lb = 0.0
      ub = 1.0
      prob = QuadratureProblem(integrand,lb,ub,x)
      out = evaluatef0(x) + x[end]*solve(prob,CubatureJLh(),reltol=1e-6,abstol=1e-6)[1]
      return out
  end

  log_pdfCt(x) = log_pdf(Ct(x)) + log(C.I.g(ForwardDiff.gradient(z->C.I.f.f(z),x)[end]))

  @inbounds for i=1:Ne
    @show norm(ForwardDiff.gradient(log_pdfCt, member(ens,i)) - dxlogC[i,:])<
  end

norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 2.2887833992611187e-16
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 6.938893903907228e-18
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 5.594315114139762e-17
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 2.7755575615628914e-17
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 1.3092278833360675e-16
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 0.0
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 6.684427777288335e-16
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 4.163336342344337e-17
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 2.482534153247273e-16
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 1.1102230246251565e-16
norm(ForwardDiff.gradient(log_pdfCt, member(ens, i)) - dxlogC[i, :]) = 2.22044604925

### Test Hessian of the log π(x)

In [30]:
Nx = 2
Ne = 8
m = 10

X = randn(Nx, Ne)

idx = [0 0; 0 1; 1 0; 1 1; 1 2]
coeff =   [0.6285037650645056;
 -0.4744029092496623;
  1.1405280011620331;
 -0.7217760771894809;
  0.11855056306742319];

C = MapComponent(m, Nx, idx, coeff);

In [34]:
dψ_basis = grad_x_basis(C.I.f.f, X)

8×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  -0.207296    -0.130929     0.00117735
 0.0  0.0  -0.216523    -0.134105     0.0375507
 0.0  0.0   0.00618292   0.00360034   0.00205305
 0.0  0.0  -0.199714    -0.0830991   -0.107373
 0.0  0.0   0.209274     0.0991538   -0.106331
 0.0  0.0  -0.236677    -0.10036     -0.126702
 0.0  0.0  -0.270316    -0.120917     0.142049
 0.0  0.0  -0.0785179   -0.0495593    0.00259814

[:, :, 2] =
 0.0   0.00283979  0.0   0.00155384  0.345581
 0.0   0.0867129   0.0   0.0465448   0.31942
 0.0  -0.166026    0.0  -0.104855    0.307967
 0.0  -0.268816    0.0  -0.149159    0.0381479
 0.0   0.254048    0.0   0.138466    0.109751
 0.0  -0.26767     0.0  -0.064575    0.0207737
 0.0   0.262746    0.0   0.0959916   0.0506544
 0.0   0.0165449   0.0   0.0102847   0.391819

In [41]:
dxlogDJ = zeros(Ne, Nx)

ψoffdψxd = grad_xd(C.I.f.f, X)

8-element Array{Float64,1}:
  0.03850011681624252
 -0.036864339594153354
  0.19095459274793916
  0.23970874743411022
 -0.20745155095311177
  0.17605463803346075
 -0.18792702339659972
  0.03117818085903034

In [50]:
grad_x_grad_xd(C.I.f.f, X, C.I.f.f.idx)

8×2 Array{Float64,2}:
 -0.0150962    0.275081
 -0.00172344   0.272766
  0.00109835   0.191537
 -0.0403773   -0.00821251
 -0.0333779    0.127178
 -0.0481415    0.0110568
  0.0468207    0.0774556
 -0.00492957   0.294571

In [51]:
hess_xd(C.I.f.f, X)

8-element Array{Float64,1}:
  0.27508054698884865
  0.2727658334063561
  0.19153741860252596
 -0.008212514607585927
  0.12717812747627785
  0.01105679171823458
  0.07745559049870646
  0.29457111787580165

In [52]:
dxdxkψx_t = ForwardDiff.hessian(C.I.f.f, X[:,1])

2×2 Array{Float64,2}:
 -0.13341    -0.0150962
 -0.0150962   0.275081

In [ ]:
evaluate()

In [16]:
log_pdf(C, X)

8-element Array{Float64,1}:
 -1.890933371333304
 -1.0774421758804786
 -1.3812358207093065
 -2.756566903253647
 -2.2764943782683207
 -3.268114225886461
 -0.9741216255311081
 -1.442529469931936

In [18]:
Cx = evaluate(C, X)

8-element Array{Float64,1}:
 -1.3182090265735815
  0.5518744832820461
  1.0207836492619973
  1.969508900840876
  1.7095646000932772
  2.2193925767103297
  0.06731649217667446
  1.0856697228586412

In [22]:
getidx(C)

5×2 Array{Int64,2}:
 0  0
 0  1
 1  0
 1  1
 1  2

In [21]:
getcoeff(C)

5-element Array{Float64,1}:
  0.6285037650645056
 -0.4744029092496623
  1.1405280011620331
 -0.7217760771894809
  0.11855056306742319

In [23]:
dψ = grad_x_basis(C.I.f.f, X)

8×5×2 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0   0.107478    0.0190908  -0.043005
 0.0  0.0   0.0865389   0.0541193  -0.0107887
 0.0  0.0   0.138011    0.0855902   0.0231549
 0.0  0.0  -0.270698   -0.113793   -0.145219
 0.0  0.0  -0.259926   -0.128072   -0.127645
 0.0  0.0  -0.225594   -0.0869868  -0.122217
 0.0  0.0  -0.266312   -0.155186    0.0881003
 0.0  0.0   0.261878    0.157944    0.0678763

[:, :, 2] =
 0.0   0.200064   0.0   0.122532   -0.167233
 0.0   0.0623342  0.0   0.0386104   0.379666
 0.0  -0.0838875  0.0  -0.0502452   0.357862
 0.0  -0.268231   0.0  -0.105483    0.0306983
 0.0  -0.24554    0.0  -0.112246    0.113373
 0.0  -0.270878   0.0  -0.142266    0.00262786
 0.0   0.165408   0.0   0.0714071   0.211025
 0.0  -0.129595   0.0  -0.0583847   0.246624

In [24]:
d2ψ = hess_x_basis(C.I.f.f, X)

8×5×2×2 Array{Float64,4}:
[:, :, 1, 1] =
 0.0  0.0  -0.287372   -0.0510446    0.114986
 0.0  0.0  -0.297614   -0.18612      0.0371031
 0.0  0.0  -0.267679   -0.166006    -0.0449099
 0.0  0.0  -0.0102897  -0.00432547  -0.00552002
 0.0  0.0  -0.0807765  -0.0398007   -0.0396678
 0.0  0.0  -0.1657     -0.0638922   -0.089769
 0.0  0.0  -0.0515665  -0.030049     0.0170591
 0.0  0.0  -0.0730321  -0.0440471   -0.0189292

[:, :, 2, 1] =
 0.0   0.0   0.0   0.0215025   -0.0293467
 0.0   0.0   0.0   0.00539434   0.0530439
 0.0  -0.0   0.0  -0.0115774    0.0824581
 0.0  -0.0  -0.0   0.0726096   -0.0211313
 0.0  -0.0  -0.0   0.0638224   -0.0644633
 0.0  -0.0  -0.0   0.0611086   -0.00112877
 0.0   0.0  -0.0  -0.0440502   -0.130179
 0.0  -0.0   0.0  -0.0339381    0.143359

[:, :, 1, 2] =
 0.0   0.0   0.0   0.0215025   -0.0293467
 0.0   0.0   0.0   0.00539434   0.0530439
 0.0  -0.0   0.0  -0.0115774    0.0824581
 0.0  -0.0  -0.0   0.0726096   -0.0211313
 0.0  -0.0  -0.0   0.0638224   -0.0644633
 0.0  -

### Integration

In [20]:
Nx = 100
Ny = 50
m = 20
Ne = 400
X = randn(Nx, Ne).*randn(Nx, Ne) #.+ 0.5*randn(Nx).*cos.(randn(Nx, Ne) .* randn(Nx, Ne))

# L = LinearTransform(X)
# transform!(L, X);
# C = MapComponent(m, Nx)
# @time C, _ = optimize(C, X, 10)

In [24]:
M = HermiteMap(m, X);

In [25]:
@time M_noqr = optimize(M, X, 10; P = serial, start = Ny+1);

  9.550136 seconds (42.61 M allocations: 13.653 GiB, 7.59% gc time)


In [26]:
@time M_qr = optimize(M, X, 10; P = serial, withqr = true, start = Ny+1);

 18.997304 seconds (45.81 M allocations: 21.111 GiB, 4.17% gc time)


In [31]:
function timing()
    M = HermiteMap(m, X)
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = false, start = Ny+1)
    end
    
    @btime begin 
        optimize($M, $X, $10; P = serial, withqr = true, start = Ny+1)
    end
end

timing (generic function with 1 method)

In [32]:
timing();

  11.257 s (42606958 allocations: 13.65 GiB)
  21.346 s (45813192 allocations: 21.12 GiB)


In [11]:
timing();

  14.597 s (42433511 allocations: 13.37 GiB)


In [29]:
timing();

  973.450 ms (1054425 allocations: 1.43 GiB)


In [14]:
methods(update_component)

# 1 method for generic function "update_component":
[1] update_component(C::MapComponent, X, reduced_margin::Array{Int64,2}, S::Storage) in AdaptiveTransportMap at /media/mat/HDD/AdaptiveTransportMap/src/mapcomponent/greedyfit.jl:249

In [30]:
function timing()
    @btime begin 
        M = HermiteMap($m, $X)
        optimize($M, $X, $10; P = thread)
    end
end

timing (generic function with 1 method)

In [88]:
timing();

  297.730 ms (1000725 allocations: 1.35 GiB)


### Test optimization with QR

In [45]:
Nx = 3
Ne = 8
m = 20

idx = [0 0 0; 0 0 1; 0 1 0; 0 1 1; 0 1 2; 1 0 0]


Nψ = 6
coeff = [ 0.20649582065364197;
         -0.5150990160472986;
          2.630096893080717;
          1.13653076177397;
          0.6725837371023421;
         -1.3126095306624133]
C = MapComponent(m, Nx, idx, coeff; α = 1e-6);

Ne = 100


# The QR decomposition is not unique!

X = randn(Nx, Ne) .* randn(Nx, Ne) + cos.(randn(Nx, Ne)) .* exp.(-randn(Nx, Ne).^2)

L = LinearTransform(X)
transform!(L, X)
S = Storage(C.I.f, X)
F = QRscaling(S)
newidx = [1 1 1]

Snew = update_storage(S, X, newidx)
Fupdated = updateQRscaling(F, Snew)

Fnew = QRscaling(Snew)

@test norm(Fupdated.D - Fnew.D)<1e-8
@test norm(Fupdated.Dinv - Fnew.Dinv)<1e-8

Test Passed